In [3]:
import os
from dotenv import load_dotenv

load_dotenv(override=True)
DASHSCOPE_API_KEY = os.getenv("DASHSCOPE_API_KEY")

In [4]:
from langchain.chat_models import init_chat_model

model = init_chat_model(
    model="deepseek-v3",
    model_provider="openai",
    api_key=DASHSCOPE_API_KEY,
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
)

In [ ]:
from langchain.output_parsers import ResponseSchema, StructuredOutputParser
from langchain.prompts import PromptTemplate

schemas = [
    ResponseSchema(name="name", description="用户的姓名"),
    ResponseSchema(name="age", description="用户的年龄"),
]

parser = StructuredOutputParser.from_response_schemas(schemas)

prompt = PromptTemplate.from_template(
    "请根据一下用户信息，返回 JSON 格式：\n{input}\n\n{format_instructions}"
)

chain = (
    prompt.partial(format_instructions=parser.get_format_instructions())
    | model
    | parser
)

result = chain.invoke({"input": "用户叫李雷，今年25岁，是一名工程师"})
print(result)

{'name': '李雷', 'age': '25'}


In [6]:
parser.get_format_instructions()

'The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":\n\n```json\n{\n\t"name": string  // 用户的姓名\n\t"age": string  // 用户的年龄\n}\n```'

In [9]:
news_gen_prompt = PromptTemplate.from_template("请根据一下新闻标题撰写一段简短的新闻内容（100字以内）：\n\n标题：{title}");

news_chain = news_gen_prompt | model

schemas = [
    ResponseSchema(name="time", description="事件发生的时间"),
    ResponseSchema(name="location", description="事件发生的地点"),
    ResponseSchema(name="event", description="发生的具体事件"),
]
parser = StructuredOutputParser.from_response_schemas(schemas)

summary_prompt = PromptTemplate.from_template(
    "请从下面这段新闻内容中提取关键信息，并返回结构化JSON格式：\n\n{news}\n\n{format_instructions}"
)

summary_chain = (
    summary_prompt.partial(format_instructions=parser.get_format_instructions())
    | model
    | parser
)

full_chain = news_chain | summary_chain

result = full_chain.invoke({"title": "苹果公司在加州发布新款AI芯片"})
print(result)

{'time': '今日', 'location': '加州总部', 'event': '苹果公司发布全新AI芯片，预计将大幅提升其设备的人工智能性能。这款芯片采用先进制程工艺，专为机器学习任务优化，未来或应用于iPhone、Mac等产品线。分析师指出，此举标志着苹果在AI领域的重大突破，有望增强其在全球科技竞争中的优势。发布会未透露具体量产时间及商用细节。'}


In [10]:
from langchain_core.runnables import RunnableLambda

def debug_print(x):
    print("中间结果（新闻正文）：", x)
    return x

debug_node = RunnableLambda(debug_print)

full_chain = news_chain | debug_node | summary_chain

result = full_chain.invoke({"title": "苹果公司在加州发布新款AI芯片"})
print(result)

中间结果（新闻正文）： content='苹果公司近日在加州总部发布全新AI芯片M4，性能提升显著，专为下一代MacBook Pro及iPad Pro设计。该芯片采用3纳米制程，集成更强神经网络引擎，支持实时AI任务处理。苹果称其能效比提升50%，将大幅增强设备在图像生成、语音识别等场景的表现。新品预计年内搭载于多款终端上市，进一步巩固其在高端移动计算市场的竞争力。（99字）' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 92, 'prompt_tokens': 28, 'total_tokens': 120, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'deepseek-v3', 'system_fingerprint': None, 'id': 'chatcmpl-29387276-ac45-9581-9501-ef012fce6e4e', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None} id='run--e81e40e0-4eb6-4316-8a12-16da9fdd221a-0' usage_metadata={'input_tokens': 28, 'output_tokens': 92, 'total_tokens': 120, 'input_token_details': {}, 'output_token_details': {}}
{'time': '近日', 'location': '加州总部', 'event': '苹果公司发布全新AI芯片M4，性能提升显著，专为下一代MacBook Pro及iPad Pro设计。该芯片采用3纳米制程，集成更强神经网络引擎，支持实时AI任务处理。苹果称其能效比提升50%，将大幅增强设备在图像生成、语音识别等场景的表现。新品预计年内搭载于多款终端上市，进一步巩固其在高端移动计算市场的竞争力。'}
